# Accessing German Historical Newspapers using the Gemini Pro Model
## Example: Article Extraction

*Notebook created by Sarah Oberbichler (oberbichler@ieg-mainz.de)*

This notebook shows how LLMs can be used to support research with historical newspapers. In this example, the Gemini pro model is used to extract articles on earthquakes in OCR'd historical newspapers pages.

Article segmentation for historical newspapers can be based on layout information and graphical elements (image) as well as on textual context (data). While the former is very challenging due to the changing and complex layouts of historical newspapers, the latter seems to be especially promising for topic-specific corpus building. Qualitative research relies on correctly separated articles. An article, in this context, is defined as a coherent text covering a specific topic, no more and no less.



### 1.   Query the German Historical Newspaper Portal

German historical newspapers from the German Digital Library can be accessed via the DDB-API. This API is open access and allows to query the Historical Newspapers available in the German Newspaper Portal ([Deutsches Zeitungsportal](https://https://www.deutsche-digitale-bibliothek.de/newspaper)). An instruction, provided by the German Newspaper Portal, can be found [here](https://https://deepnote.com/app/karl-kragelin-b83c/Zeitungsportal-API-d9224dda-8e26-4b35-a6d7-40e9507b1151).

Python > 3.8 is required

In [1]:
# @markdown ####  Launch this cell and get access to the API of the Newspaper Portal from the German Digital Library
!pip install ddbapi

In [2]:
# @markdown ####  Import the necessary packages
import pandas as pd
from ddbapi import zp_issues, zp_pages, list_column, filter

In [3]:

# @markdown ### Possible kwargs for the functions are:
# @markdown - language: Use ISO Codes, currently ger, eng, fre, spa
# @markdown - place_of_distribution: Search inside "Verbreitungsort"
# @markdown - use a list for multiple search-words
# @markdown - publication_date: Get newspapers by publication date.
# @markdown - zdb_id: Search by ZDB-ID
# @markdown - provider: Search by Data Provider
# @markdown - paper_title: Search inside the title of the Newspaper
# @markdown - plainpagefulltex: search inside the OCR
# Get the data
df = zp_pages(
    publication_date='[1850-01-01T12:00:00Z TO 1851-12-31T12:00:00Z]',
    plainpagefulltext=["Erdbeben*"],
    #paper_title='Deutsche allgemeine Zeitung'
    )
df

https://api.deutsche-digitale-bibliothek.de/search/index/newspaper-issues/select?rows=1000&sort=id+ASC&q=type%3Apage+AND+publication_date%3A%22%5B1850-01-01T12%3A00%3A00Z%5C+TO%5C+1851-12-31T12%3A00%3A00Z%5D%22+AND+%28plainpagefulltext%3AErdbeben%2A%29&cursorMark=%2A
Got 767 items.


,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext
0,22NYUNGOY64IXJJMTMPKYR7UADDF6GVH-ALTO8578354_D...,3,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1851-05-17 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],d9797b92-931d-4a4b-b99d-21a041a30d1d,[/data/altos/22/NY/22NYUNGOY64IXJJMTMPKYR7UADD...,ALTO8578354_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,piemontesisch = schweizerischen Telegraphen = ...
1,24I6NCIJKY6FXQ6WLXRZ642KPYXATVNO-ALTO2890815_D...,2,Neuwiedische Nachrichten,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2731244-6,1851-07-01 12:00:00,[Neuwied],[ger],480ccbcd-b912-40fe-aa7f-60c2b1c3fb31,[/data/altos/24/I6/24I6NCIJKY6FXQ6WLXRZ642KPYX...,ALTO2890815_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Stegen Haussuchung gehalten , und zwar , wie e..."
2,26JVYAU3S5GIKD2P6V4ZD6SPZRYVGH5J-ALTO3085102_D...,1,Täglicher Anzeiger für Berg und Mark,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2724665-6,1850-12-01 12:00:00,"[Wuppertal, Elberfeld, Wuppertal-Elberfeld]",[ger],cccc47ad-8958-470b-bedd-277b6cc43b82,[/data/altos/26/JV/26JVYAU3S5GIKD2P6V4ZD6SPZRY...,ALTO3085102_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"# # # int & amp N . 286 Elberfeld , Sonntag de..."
3,26ZR5I7GWW3DAFUXMVRLMLBY3PJJHO6X-FILE_0003_DDB...,3,Staats- und gelehrte Zeitung des Hamburgischen...,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Car...,2260914-3,1851-10-23 12:00:00,[Hamburg],[ger],0459305c-9a65-429b-8ccf-c36487f6d94c,[/data/altos/26/ZR/26ZR5I7GWW3DAFUXMVRLMLBY3PJ...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,»HP h* dis ^iatS- »iFcrä ’t'l'llfö sst'sspf UH...
4,27PZTJBLDL6AHEAFKPBUTAMHVVSGDVSM-FILE_0003_DDB...,3,Dresdner Journal und Anzeiger,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2889118-1,1851-05-05 12:00:00,[Dresden],[ger],075be8b6-4850-4b88-876e-0725946cf523,[/data/altos/27/PZ/27PZTJBLDL6AHEAFKPBUTAMHVVS...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,991 worden sind. Der preußische General Brandt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,ZTCTXNF7CGGRMPBQT7QPL3WU2KYNNF5Y-FILE_0004_DDB...,4,Riedlinger Zeitung : Tag- und Anzeigeblatt für...,VNHXUCEEKHOUSYH4NVOUBHJGSRMOGK7J,Württembergische Landesbibliothek,3123872-5,1851-07-09 12:00:00,[Riedlingen],[ger],d2219472-3416-4cd2-9601-62f1ff519203,[/data/altos/ZT/CT/ZTCTXNF7CGGRMPBQT7QPL3WU2KY...,FILE_0004_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,220 noch nicht entmuthigt. Diese standen ja ni...
763,ZWTKOXXZ777DIG2CQI4MNFRWDVRFK66Z-ALTO8577905_D...,10,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1851-03-25 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],9f5d0a66-99dd-4ca7-a83e-142b4a54a27e,[/data/altos/ZW/TK/ZWTKOXXZ777DIG2CQI4MNFRWDVR...,ALTO8577905_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"erwarten , daß das neue Ministerium die Aufgab..."
764,ZXJYZGE3JVRSOM7QEL5PIZG64D34M4WD-FILE_0003_DDB...,3,Altonaer Mercur : Zeitung und Intelligenzblatt...,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Car...,3033383-0,1851-03-16 12:00:00,"[Hamburg, Hamburg-Altona, Altona, Lauenburg/Elbe]",[ger],b51b1ede-d7b8-4417-a382-f77164a476ce,[/data/altos/ZX/JY/ZXJYZGE3JVRSOM7QEL5PIZG64D3...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,ftht Me Einrichtung einer Prüfungskommission f...
765,ZXOSMVED5EUTG2KPYNWXNWVJHPHSYVNK-ALTO8832491_D...,2,Düsseldorfer Journal und Kreisblatt,A5MCTVQDBFJTFJDZRTIZX5W6YKSHWOVC,Heinrich-Heine-Universität Düsseldorf. Univers...

In [4]:
# @markdown #### We can narrow down the text surrounding the keyword in order to reduce the input tokens for the model. Choose the size of the context window here:

context_window = 5000 # @param {type:"number"}
def extract_context(keyword, text, window_size=context_window):
    index = text.find(keyword)
    if index == -1:
        return "Keyword not found in text."

    start_index = max(0, index - window_size)
    end_index = min(len(text), index + len(keyword) + window_size)

    context = text[start_index:end_index]

    return context


# Extract context for each row
contexts = []
for index, row in df.iterrows():
    text = row['plainpagefulltext']
    keyword = "rdbeben"  # You can modify this
    context = extract_context(keyword, text)
    contexts.append(context)

# Add the context to the dataframe
df['context'] = contexts

# Print the dataframe with context
df.head()

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext,context
0,22NYUNGOY64IXJJMTMPKYR7UADDF6GVH-ALTO8578354_D...,3,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1851-05-17 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],d9797b92-931d-4a4b-b99d-21a041a30d1d,[/data/altos/22/NY/22NYUNGOY64IXJJMTMPKYR7UADD...,ALTO8578354_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,piemontesisch = schweizerischen Telegraphen = ...,allenen Unweiters . — Die mininisterielle „ Ze...
1,24I6NCIJKY6FXQ6WLXRZ642KPYXATVNO-ALTO2890815_D...,2,Neuwiedische Nachrichten,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2731244-6,1851-07-01 12:00:00,[Neuwied],[ger],480ccbcd-b912-40fe-aa7f-60c2b1c3fb31,[/data/altos/24/I6/24I6NCIJKY6FXQ6WLXRZ642KPYX...,ALTO2890815_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Stegen Haussuchung gehalten , und zwar , wie e...",tig . Namentlich hat sie sich in den jüngsten ...
2,26JVYAU3S5GIKD2P6V4ZD6SPZRYVGH5J-ALTO3085102_D...,1,Täglicher Anzeiger für Berg und Mark,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2724665-6,1850-12-01 12:00:00,"[Wuppertal, Elberfeld, Wuppertal-Elberfeld]",[ger],cccc47ad-8958-470b-bedd-277b6cc43b82,[/data/altos/26/JV/26JVYAU3S5GIKD2P6V4ZD6SPZRY...,ALTO3085102_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"# # # int & amp N . 286 Elberfeld , Sonntag de...","ommen ; , lieber einen Kampf auf Tod und Leben..."
3,26ZR5I7GWW3DAFUXMVRLMLBY3PJJHO6X-FILE_0003_DDB...,3,Staats- und gelehrte Zeitung des Hamburgischen...,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Car...,2260914-3,1851-10-23 12:00:00,[Hamburg],[ger],0459305c-9a65-429b-8ccf-c36487f6d94c,[/data/altos/26/ZR/26ZR5I7GWW3DAFUXMVRLMLBY3PJ...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,»HP h* dis ^iatS- »iFcrä ’t'l'llfö sst'sspf UH...,"er. Der Sü-ele, der von Allem, was Villault an..."
4,27PZTJBLDL6AHEAFKPBUTAMHVVSGDVSM-FILE_0003_DDB...,3,Dresdner Journal und Anzeiger,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2889118-1,1851-05-05 12:00:00,[Dresden],[ger],075be8b6-4850-4b88-876e-0725946cf523,[/data/altos/27/PZ/27PZTJBLDL6AHEAFKPBUTAMHVVS...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,991 worden sind. Der preußische General Brandt...,"ein wir-, daß Alles, waS nicht gerade entschie..."


In [5]:
# @markdown #### Save the results as Excel file
df.to_excel('name.xlsx', index=False)


## Setting up the requirements for the Gemini model

Gemini is a family of generative AI models that helps generate content and solve problems. These models are designed and trained to handle both text and images as input.

In [6]:
!pip install -q -U google-generativeai

In [7]:
import google.generativeai as genai

In [8]:
# @markdown ##### Get an API key at https://aistudio.google.com/app/prompts/new_chat, activate the pay as you go mode, and add the key to the secrets in this colab notebook (right bar). Name the key in secrets GOOGLE_API_KEY and add the key under value.
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
# @markdown ##### Set up the model. Find a list of the available GEMINI models here: https://ai.google.dev/gemini-api/docs/models/gemini. The safety settings can be blocked in order to have no restrictions with your data.
model = genai.GenerativeModel('gemini-1.5-pro')

# Extract Articles

To extract articles on earthquakes, it is essential to formulate a precise prompt that specifies the articles should be extracted in their original form without translations or corrections. A guide on how to write effective prompts can be found also [here](https://https://support.google.com/a/users/answer/14200040?hl=en).

Depending on the size of the dataframe, it can take a while to load.

In [10]:
df=df[:20]

In [11]:
import json
def separate_articles(newspaper_page):
    # Define the prompt for separating articles

    response = model.generate_content(
    [f"Bitte separiere NUR Berichte zu Erdbeben in ihrer ungeänderten deutschen Originalform, keine Änderungen, Zusätze oder zusätzliche Beschreibungen\n\n{newspaper_page}\n\n---\n\ ."])
    articles=response.text
    return articles



# Create an empty list to store the separated articles
separated_articles = []

# Assuming `df` is your dataframe
# Loop through each row in the dataframe
for index, row in df.iterrows():
    # Extract the text of the newspaper page from the current row
    newspaper_page = row['context']

    # Separate articles for the current newspaper page
    articles = separate_articles(newspaper_page)


    # Append the articles to the list (assuming articles is a string; you might need to adjust if it's a list or another structure)
    separated_articles.append(articles)

# Add the list of separated articles as a new column 'article' in the dataframe
df['article'] = separated_articles

# Print the modified dataframe
df


<ipython-input-11-d58c6bb489e7>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article'] = separated_articles


,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext,context,article
0,22NYUNGOY64IXJJMTMPKYR7UADDF6GVH-ALTO8578354_D...,3,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1851-05-17 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],d9797b92-931d-4a4b-b99d-21a041a30d1d,[/data/altos/22/NY/22NYUNGOY64IXJJMTMPKYR7UADD...,ALTO8578354_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,piemontesisch = schweizerischen Telegraphen = ...,allenen Unweiters . — Die mininisterielle „ Ze...,"Eine frühere Citadelle , vermuthlich das Caste..."
1,24I6NCIJKY6FXQ6WLXRZ642KPYXATVNO-ALTO2890815_D...,2,Neuwiedische Nachrichten,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2731244-6,1851-07-01 12:00:00,[Neuwied],[ger],480ccbcd-b912-40fe-aa7f-60c2b1c3fb31,[/data/altos/24/I6/24I6NCIJKY6FXQ6WLXRZ642KPYX...,ALTO2890815_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Stegen Haussuchung gehalten , und zwar , wie e...",tig . Namentlich hat sie sich in den jüngsten ...,Am 2 . April gegen 6½ Uhr Morgens wurde Valpa ...
2,26JVYAU3S5GIKD2P6V4ZD6SPZRYVGH5J-ALTO3085102_D...,1,Täglicher Anzeiger für Berg und Mark,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2724665-6,1850-12-01 12:00:00,"[Wuppertal, Elberfeld, Wuppertal-Elberfeld]",[ger],cccc47ad-8958-470b-bedd-277b6cc43b82,[/data/altos/26/JV/26JVYAU3S5GIKD2P6V4ZD6SPZRY...,ALTO3085102_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"# # # int & amp N . 286 Elberfeld , Sonntag de...","ommen ; , lieber einen Kampf auf Tod und Leben...",Der Text enthält keinen Bericht über Erdbeben. \n
3,26ZR5I7GWW3DAFUXMVRLMLBY3PJJHO6X-FILE_0003_DDB...,3,Staats- und gelehrte Zeitung des Hamburgischen...,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Car...,2260914-3,1851-10-23 12:00:00,[Hamburg],[ger],0459305c-9a65-429b-8ccf-c36487f6d94c,[/data/altos/26/ZR/26ZR5I7GWW3DAFUXMVRLMLBY3PJ...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,»HP h* dis ^iatS- »iFcrä ’t'l'llfö sst'sspf UH...,"er. Der Sü-ele, der von Allem, was Villault an...",Zn port of Spain (Jamaika) war wieder ein leic...
4,27PZTJBLDL6AHEAFKPBUTAMHVVSGDVSM-FILE_0003_DDB...,3,Dresdner Journal und Anzeiger,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2889118-1,1851-05-05 12:00:00,[Dresden],[ger],075be8b6-4850-4b88-876e-0725946cf523,[/data/altos/27/PZ/27PZTJBLDL6AHEAFKPBUTAMHVVS...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,991 worden sind. Der preußische General Brandt...,"ein wir-, daß Alles, waS nicht gerade entschie...","Palermo, 14. April. Ein Erdbeben hat in Messin..."
5,2AFZAFNOG2CCXPTK2O4O4UG6H7USN2OD-FILE_0005_DDB...,5,Schwäbischer Merkur : mit Schwäbischer Kronik ...,VNHXUCEEKHOUSYH4NVOUBHJGSRMOGK7J,Württembergische Landesbibliothek,2751625-8,1851-09-06 12:00:00,[Stuttgart],[ger],ddff4a11-d067-45f2-b15b-dff431bb4d25,[/data/altos/2A/FZ/2AFZAFNOG2CCXPTK2O4O4UG6H7U...,FILE_0005_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"1207 auswärtigen Angelegenheiten, hat seine En...","1207 auswärtigen Angelegenheiten, hat seine En...","Den 24 August. So eben höre ich, daß ein neues..."
6,2EHKK4YK2LPQMJ5E4LK43KG6UKKVNXLL-FILE_0002_DDB...,2,Intelligenz- und Wochenblatt für Frankenberg m...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,3107190-9,1851-04-30 12:00:00,"[Frankenberg/Sa., Sachsenburg (Frankenberg, Sa...",[ger],40525de4-83d3-453b-80bb-cf2fc759892e,[/data/altos/2E/HK/2EHKK4YK2LPQMJ5E4LK43KG6UKK...,FILE_0002_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"auch Alles das erlogen ist, was man Son der Mi...","auch Alles das erlogen ist, was man Son der Mi...",KomMÄden 2

In [12]:
df.to_excel('name_2.xlsx', index=False)